When we cannot afford to sample the quantity of interest many times at every design within an optimization, we can use surrogate models instead. Here we will show you how to use the simple polynomial surrogates provided with python, as well as third party surrogates.

Firstly using the effective-quadratures package [Seshadri, P. and Parks, G. (2017) Effective-Quadratures (EQ): Polynomials for Computational Engineering Studies. The Open Journal. http://dx.doi.org/10.21105/joss.0016], (also see http://www.effective-quadratures.org/). 

Secondly using pyKriging [pyKriging 0.5281/zenodo.593877] (also see http://pykriging.com/).

To begin with we'll use the polynomial surrogate that comes with horsetail matching. 

In [6]:
from horsetailmatching import HorsetailMatching, UncertainParameter
from horsetailmatching.demoproblems import TP1, TP2
from horsetailmatching.surrogates import polySurrogate

The polySurrogate function takes a numpy array of input values of size (num_samples, num_input_dimensions), and a array of output values of size (num_samples). It returns a function that will return a predicted output value at any input value. Using this with horsetail matching is then as simple as passing this function to the surrogate argument:

In [9]:
uparams = [UncertainParameter('uniform'), UncertainParameter('gaussian')]
theHM = HorsetailMatching(TP1, uparams, surrogate=polySurrogate)

print(theHM.evalMetric([0, 1]))

21.2731701322


As with everything in horsetail matching, if this is all we do then it will do its best to automatically decide the rest. In this case the other key thing we need is the values of the uncertain parameters at which to evaluate the true qoi to give the values the surrogate model is fitted to. 